In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

# from torchsummary import summary

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as data

from resnet import BasicBlock, ResNet, Bottleneck
from train_eval_util import train, evaluate, calculate_accuracy, epoch_time
from getCIFAR10 import train_data, valid_data, test_data

import time

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')  
else:
    device = torch.device('cpu')

print(f"Selected device: {device}")

Selected device: cuda


In [5]:
BATCH_SIZE = 128

train_iterator = DataLoader(train_data, batch_size= BATCH_SIZE, shuffle=True)
valid_iterator =  DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False)
test_iterator =  DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [6]:
def total_layers(model):
    return sum([1 for m in model.modules() if isinstance(m, nn.Conv2d)]) - 2

def total_params(model):
     return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [7]:
model = ResNet(BasicBlock, [1, 1, 1, 1]).to(device)

print(f"Total number of layers: {total_layers(model)}")
print(f"Total number of parameters: {total_params(model)}")

# summary(model)


Total number of layers: 10
Total number of parameters: 4903242


In [8]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

In [9]:
num_epochs = 100
train_acc_history = []
train_loss_history = []
valid_acc_history = []
valid_loss_history = []

for epoch in range(num_epochs):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
        
    end_time = time.time()

        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

    train_acc_history.append(train_acc)
    train_loss_history.append( train_loss)
    valid_acc_history.append(valid_acc)
    valid_loss_history.append(valid_loss)

Epoch: 01 | Epoch Time: 0m 23s
	Train Loss: 1.339 | Train Acc: 50.90%
	 Val. Loss: 1.294 |  Val. Acc: 55.70%
Epoch: 02 | Epoch Time: 0m 20s
	Train Loss: 0.930 | Train Acc: 66.73%
	 Val. Loss: 1.030 |  Val. Acc: 65.37%
Epoch: 03 | Epoch Time: 0m 20s
	Train Loss: 0.751 | Train Acc: 73.56%
	 Val. Loss: 0.748 |  Val. Acc: 72.99%
Epoch: 04 | Epoch Time: 0m 21s
	Train Loss: 0.636 | Train Acc: 77.87%
	 Val. Loss: 0.638 |  Val. Acc: 78.50%
Epoch: 05 | Epoch Time: 0m 20s
	Train Loss: 0.567 | Train Acc: 80.13%
	 Val. Loss: 0.497 |  Val. Acc: 82.52%
Epoch: 06 | Epoch Time: 0m 20s
	Train Loss: 0.503 | Train Acc: 82.26%
	 Val. Loss: 0.514 |  Val. Acc: 81.97%
Epoch: 07 | Epoch Time: 0m 19s
	Train Loss: 0.462 | Train Acc: 83.99%
	 Val. Loss: 0.496 |  Val. Acc: 83.30%
Epoch: 08 | Epoch Time: 0m 19s
	Train Loss: 0.420 | Train Acc: 85.35%
	 Val. Loss: 0.423 |  Val. Acc: 85.02%
Epoch: 09 | Epoch Time: 0m 20s
	Train Loss: 0.388 | Train Acc: 86.66%
	 Val. Loss: 0.434 |  Val. Acc: 85.66%
Epoch: 10 | Epoch T